In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
import os
import random
import glob
import pickle
import tqdm
from model import *

Using TensorFlow backend.


In [2]:
def generator(IDs, yields, batch_size, cutoff=None):
    import numpy as np
    import random
    
 # Create empty arrays to contain batch of features and labels#

    if cutoff != None:
        batch_features = np.zeros((batch_size, cutoff, 1, 128, 11))
        batch_yields = np.zeros((batch_size))
        while True:
            for i in range(batch_size):
                # choose random index in features
                index = random.choice(range(len(IDs)))
                ID = IDs[index]
                if np.sum(np.isnan(np.load('data/PROCESSED_III/' + ID + '.npy'))) == 0:
                    batch_features[i, :, :, :, :] = np.load('data/PROCESSED_III/' + ID + '.npy')[:cutoff, :, :, :]
                    #print('yes', ID)
                    batch_yields[i] = yields[ID]
                else:
                    print('no', ID)
                    
            yield batch_features, batch_yields
                    
    else:
        batch_features = np.zeros((batch_size, 38, 1, 128, 11))
        batch_yields = np.zeros((batch_size))
        while True:
            for i in range(batch_size):
                # choose random index in features
                index = random.choice(range(len(IDs)))
                ID = IDs[index]
                if np.sum(np.isnan(np.load('data/PROCESSED_III/' + ID + '.npy'))) == 0:
                    batch_features[i, :, :, :, :] = np.load('data/PROCESSED_III/' + ID + '.npy')
                    #print('yes', ID)
                    batch_yields[i] = yields[ID]
                else:
                    print('no', ID)
            yield batch_features, batch_yields

In [5]:
from numpy.random import seed
seed(42)
from tensorflow import set_random_seed
set_random_seed(42)

model_list = ['CNN_LSTM', 'SepCNN_LSTM', 'CONVLSTM', 'CONV3D', 'CONVLSTM_CONV3D']

# Datasets
yields = pickle.load(open('data/yields.p', 'rb'))
y = yields

# define early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, \
                          verbose=1, mode='auto')
callbacks_list = [earlystop]

# Model Training

In [3]:
# training model on data of year 2010-2015 (6 years total)

model_list = ['CNN_LSTM', 'SepCNN_LSTM', 'CONVLSTM', 'CONV3D', 'CONVLSTM_CONV3D']

# Datasets
yields = pickle.load(open('data/yields.p', 'rb'))
y = yields
print(len(yields['train']), len(yields['validation']))

# define early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, \
                          verbose=1, mode='auto')
callbacks_list = [earlystop]

# Generators
training_generator = generator(list(y['train'].keys()), y['train'], 16)
validation_generator = generator(list(y['validation'].keys()), y['validation'], 16)

for model_name in model_list:
    rm = ResearchModels(model_name, 38, (1, 128, 11), print_model=True)
    rm.model.fit_generator(training_generator, validation_data=validation_generator, callbacks=callbacks_list,\
                               validation_steps=1353/16, samples_per_epoch=50, nb_epoch=100, verbose=0)
    rm.save(model_name)

Loading Model.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 38, 1, 128, 11)    0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 38, 4096)          18112     
_________________________________________________________________
lstm_1 (LSTM)                (None, 38, 256)           4457472   
_________________________________________________________________
dense_1 (Dense)              (None, 38, 64)            16448     
_________________________________________________________________
flatten_2 (Flatten)          (None, 2432)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2432)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 2433  

/mnt/home/phongpre/anaconda3-singularity/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/mnt/home/phongpre/anaconda3-singularity/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_data=<generator..., callbacks=[<keras.ca..., validation_steps=84.5625, verbose=0, steps_per_epoch=1, epochs=1)`


Loading Model.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 38, 1, 128, 11)    0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 38, 4096)          17430     
_________________________________________________________________
lstm_2 (LSTM)                (None, 38, 256)           4457472   
_________________________________________________________________
dense_3 (Dense)              (None, 38, 64)            16448     
_________________________________________________________________
flatten_4 (Flatten)          (None, 2432)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 2432)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 2433  

/mnt/home/phongpre/Springboard/model.py:140: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(2, 1, 1), strides=(1, 1, 1), padding="valid")`
  border_mode='valid'))


Loading Model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_4 (ConvLSTM2D)  (None, 38, 1, 128, 64)    38656     
_________________________________________________________________
conv_lst_m2d_5 (ConvLSTM2D)  (None, 38, 1, 128, 32)    24704     
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 38, 1, 128, 32)    2080      
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 38, 1, 64, 32)     0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 77824)             0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 77824)             311296    
_________________________________________________________________
dense_9 (Dense)              (None, 64)                4980800

# Model Evaluation

In [4]:
# Evalurate model on the yields of corn yields across U.S. in year 2016
from numpy.random import seed
seed(42)
from tensorflow import set_random_seed
set_random_seed(42)

test_gen = generator(list(yields['validation'].keys()), yields['validation'], len(yields['validation']))
X_test, y_test = next(test_gen)
abs_error = np.empty(len(model_list))

for i, model_name in enumerate(model_list):
    rm = ResearchModels('No model', 38, (1, 128, 11), saved_model=model_name)
    abs_error[i] = rm.model.evaluate(X_test, y_test, batch_size=16)[1]
    print('For model {}, the test mean absolute error is {:.2f}.'.format(model_name, abs_error[i]))

best_model = model_list[np.argmin(abs_error)]
print('The best model is {}.'.format(best_model))

Loading model CNN_LSTM
1353/1353 [==============================] - 16s 12ms/step
For model CNN_LSTM, the test mean absolute error is 18.85.
Loading model SepCNN_LSTM
1353/1353 [==============================] - 16s 12ms/step
For model SepCNN_LSTM, the test mean absolute error is 24.11.
Loading model CONVLSTM
1353/1353 [==============================] - 65s 48ms/step
For model CONVLSTM, the test mean absolute error is 16.20.
Loading model CONV3D
1353/1353 [==============================] - 30s 22ms/step
For model CONV3D, the test mean absolute error is 96.39.
Loading model CONVLSTM_CONV3D
1353/1353 [==============================] - 57s 42ms/step
For model CONVLSTM_CONV3D, the test mean absolute error is 36.14.
The best model is CONVLSTM.


# Optimization of batch size

In [5]:
batch_size = [8, 32, 64]
abs_error_batch = np.empty(len(batch_size))

for i, size in enumerate(batch_size):
    # Generators
    training_generator = generator(list(y['train'].keys()), y['train'], size)
    validation_generator = generator(list(y['validation'].keys()), y['validation'], size)

    rm = ResearchModels('None', 38, (1, 128, 11), saved_model=best_model)
    rm.model.fit_generator(training_generator, validation_data=validation_generator, callbacks=callbacks_list,\
                               validation_steps=1353/size, samples_per_epoch=50, nb_epoch=100, verbose=0)

    abs_error_batch[i] = rm.model.evaluate(X_test, y_test, batch_size=size)[1]
    print('For batch {}, the test mean absolute error is {:.2f}.'.format(size, abs_error_batch[i]))
    
    rm.model.save(best_model + '_' + str(size))

Loading model CONVLSTM


/mnt/home/phongpre/anaconda3-singularity/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  # Remove the CWD from sys.path while we load stuff.
/mnt/home/phongpre/anaconda3-singularity/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_data=<generator..., callbacks=[<keras.ca..., validation_steps=169.125, verbose=0, steps_per_epoch=50, epochs=100)`
  # Remove the CWD from sys.path while we load stuff.


Epoch 00009: early stopping
1353/1353 [==============================] - 66s 48ms/step
For batch 8, the test mean absolute error is 61.92.
Loading model CONVLSTM


/mnt/home/phongpre/anaconda3-singularity/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_data=<generator..., callbacks=[<keras.ca..., validation_steps=42.28125, verbose=0, steps_per_epoch=50, epochs=100)`
  # Remove the CWD from sys.path while we load stuff.


Epoch 00006: early stopping
1353/1353 [==============================] - 63s 47ms/step
For batch 32, the test mean absolute error is 17.17.
Loading model CONVLSTM


/mnt/home/phongpre/anaconda3-singularity/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_data=<generator..., callbacks=[<keras.ca..., validation_steps=21.140625, verbose=0, steps_per_epoch=50, epochs=100)`
  # Remove the CWD from sys.path while we load stuff.


Epoch 00009: early stopping
1353/1353 [==============================] - 64s 47ms/step
For batch 64, the test mean absolute error is 20.10.


# Reducing the number of frames per year

In [6]:
n_frames = [1, 5, 10, 15, 20, 25, 30, 35]
abs_error_frames = np.empty(len(n_frames))
i = 0

for frame in n_frames:
    # Generators
    print('Doing {} frames per year'.format(frame))
    training_generator = generator(list(y['train'].keys()), y['train'], 16, cutoff=frame)
    validation_generator = generator(list(y['validation'].keys()), y['validation'], 16, cutoff=frame)
    
    test_gen = generator(list(yields['validation'].keys()), yields['validation'], len(yields['validation']), cutoff=frame)
    X_test, y_test = next(test_gen)

    rm = ResearchModels('CONVLSTM', frame, (1, 128, 11))
    rm.model.fit_generator(training_generator, validation_data=validation_generator, callbacks=callbacks_list,\
                               validation_steps=1353/16, samples_per_epoch=50, nb_epoch=100, verbose=0)
    abs_error_frames[i] = rm.model.evaluate(X_test, y_test, batch_size=16)[1]
    
    rm.model.save('CONVLSTM' + '_' + str(frame))
    print('For {} frames per year, the test mean absolute error is {:.2f}.'.format(frame, abs_error_frames[i]))
    i += 1
    pickle.dump( abs_error_frames, open( "abs_error_framess.p", "wb" ) )

Doing 1 frames per year
Loading Model.


/mnt/home/phongpre/anaconda3-singularity/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  from ipykernel import kernelapp as app
/mnt/home/phongpre/anaconda3-singularity/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_data=<generator..., callbacks=[<keras.ca..., validation_steps=84.5625, verbose=0, steps_per_epoch=50, epochs=100)`
  from ipykernel import kernelapp as app


Epoch 00010: early stopping
1353/1353 [==============================] - 1s 524us/step
For 1 frames per year, the test mean absolute error is 79.35.
Doing 5 frames per year
Loading Model.
Epoch 00017: early stopping
1353/1353 [==============================] - 9s 6ms/step
For 5 frames per year, the test mean absolute error is 59.80.
Doing 10 frames per year
Loading Model.
Epoch 00008: early stopping
1353/1353 [==============================] - 17s 12ms/step
For 10 frames per year, the test mean absolute error is 56.49.
Doing 15 frames per year
Loading Model.
Epoch 00010: early stopping
1353/1353 [==============================] - 26s 19ms/step
For 15 frames per year, the test mean absolute error is 27.07.
Doing 20 frames per year
Loading Model.
Epoch 00025: early stopping
1353/1353 [==============================] - 33s 25ms/step
For 20 frames per year, the test mean absolute error is 22.55.
Doing 25 frames per year
Loading Model.
Epoch 00014: early stopping
1353/1353 [================